# Wind Speed Analysis at Dublin Airport: insights and trends
***


![redanemometer](redanemometer.jpg)

## Context and Purpose
***

- Long-Term Wind Speed Trends at Dublin Airport
- timeframes (2000-2025)

## Getting ready
***

### Importing Libraries


In [28]:
# Data frames
import pandas as pd


# Importing matplotlib for Plotting
import matplotlib.pyplot as plt

# Importing scikit-learn (sklearn) for machine learning tasks
import sklearn 

# Importing specific functions from scikit-learn (sklearn)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression



### Set up and installing

Ensure all the necessary tools are installed in Python environment:

1. Run in the terminal the following commands to install Jupyter notebook and pandas:


`pip install notebook`

`pip install pandas`

`pip install scikit-learn`

2. Check Jupyter extension is installed

3. Start the Kernel if It is not running to make sure the execution of all cells and output results.

### Loading data

In [29]:
# Loading data into notebook from csv file
data = pd.read_csv("clean_data_DublinAirport_weather.csv")

C:\Users\noemi\AppData\Local\Temp\ipykernel_18836\118780773.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("clean_data_DublinAirport_weather.csv")


### Examine dataset

Once the data frame is uploaded, the analysis can proceed.The dataset used from Met Eireann has been cleaned and organized focus on four variables:

- date: date
- airtemp: air tempeture in °C
- wdsp: Wind speed
- wddir: Wind direction

In [30]:
# Information about columns names and data types of variables
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218424 entries, 0 to 218423
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   date     218424 non-null  object 
 1   airtemp  218424 non-null  float64
 2   wdsp     218424 non-null  int64  
 3   wddir    218424 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 6.7+ MB


In [31]:
# Check for missing values
data.isnull().sum()


date       0
airtemp    0
wdsp       0
wddir      0
dtype: int64

In [32]:
# show the data
data.head()

,date,airtemp,wdsp,wddir
0,01/01/2000 01:00,3.3,7,250
1,01/01/2000 02:00,3.4,8,250
2,01/01/2000 03:00,2.2,6,240
3,01/01/2000 04:00,2.2,6,230
4,01/01/2000 05:00,1.9,7,250


In [33]:
# Statistics summary of numerical variables
data.describe()


,airtemp,wdsp
count,218424.000000,218424.000000
mean,9.857803,10.303927
std,5.025941,5.295946
min,-11.500000,0.000000
25%,6.300000,6.000000
50%,10.000000,10.000000
75%,13.500000,13.000000
max,29.100000,45.000000


## Data visualization

notes
- How much wind power in a day /year /month ?
- wind direction

#### Wind direction frequency

The following pie chart highlights the wind direction based on the cardinal points. The wind predominantly blows towards the South West and West with a slightly less frequency towards the South.

The wind direction data (in degrees) is usually given in terms of cardinal directions (e.g., North = 0 or 360 degrees, East = 90 degrees, South = 180 degrees, West = 270 degrees)

In [34]:
data['wddir'].value_counts()

wddir
240    9348
260    9272
250    8804
270    8169
230    7889
       ... 
10      764
360     726
0       143
0         8
          1
Name: count, Length: 75, dtype: int64

In [37]:
# Define the cardinal directions based on degrees
def get_cardinal_direction(degrees):
    if degrees == 0 or degrees == 360:
        return 'North'
    elif degrees == 90:
        return 'East'
    elif degrees == 180:
        return 'South'
    elif degrees == 270:
        return 'West'
    else:
        return 'Other'

# Apply the function to the 'wddir' column
data['wddir_cardinal'] = data['wddir'].apply(get_cardinal_direction)

# Show the counts of cardinal directions
print(data['wddir_cardinal'].value_counts())


wddir_cardinal
Other    208787
West       5901
East       1995
South      1007
North       734
Name: count, dtype: int64


In [ ]:
# Plotting a pie chart about the frequency of cardinal wind directions
labels = 'South West', 'West', 'South'
sizes = [7, 7, 5]
colors=['seagreen','yellowgreen','darkgreen']
hatch_patterns= ['//', 'o','..']
fig, ax = plt.subplots()
ax.pie(sizes, labels=labels, colors=colors, hatch=hatch_patterns, shadow=True,  textprops={'size': 'smaller'}, radius=0.8)
plt.title('Frequency of Cardinal Wind Directions (Single Day)')
plt.legend(labels, loc="best", fontsize="small")
plt.show

## References

- Jupyter notebook format: 

https://www.ibm.com/docs/en/watson-studio-local/1.2.3?topic=notebooks-markdown-jupyter-cheatsheet

- Jupyter notebook text formatting: 

https://www.earthdatascience.org/courses/intro-to-earth-data-science/file-formats/use-text-files/format-text-with-markdown-jupyter-notebook/

- Free images:
https://pixabay.com/

- Classify the wind direction:
https://stackoverflow.com/questions/35864909/classify-the-wind-direction-in-several-classes

***
### End